In [1]:
import pandas as pd
import numpy as np
import warnings

import statsmodels.api as sm

from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
#pd.options.display.float_format = '{:.2f}'.format
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn

In [2]:
def preprocess_data(data, features, imputer):
    data[features] = imputer.transform(data[features])
    return data

In [3]:
def train_model_statsmodels(X, y):
    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit(cov_type='HC0')
    return model

In [4]:
def create_k_folds(df, fold_n=5):
    folds = {}
    fold_size = len(df) // fold_n
    for i in range(fold_n):
        start = i * fold_size
        if i == fold_n - 1:  # In the last fold, include all remaining data
            end = len(df)
        else:
            end = start + fold_size
        folds[i] = df[start:end]
    return folds

In [5]:
def find_index(lst, target):
    for i, number in enumerate(lst):
        if number == target:
            return i
    return None

In [6]:
def train_final_model(folds_X, folds_y):
    cv_amount = len(folds_X)
    models, rmse_scores, predictions = {}, {}, {}
    rmse_total = 0
    
    for i in range(cv_amount):
        test_X, test_y = folds_X[i], folds_y[i]
        train_X = np.concatenate([folds_X[n] for n in range(len(folds_X)) if n != i])
        train_y = np.concatenate([folds_y[n] for n in range(len(folds_y)) if n != i])

        models['model_{}'.format(i)] = train_model_statsmodels(train_X, train_y)
        test_X = sm.add_constant(test_X, has_constant='add')
        y_pred = models['model_{}'.format(i)].predict(test_X)
        predictions[i] = y_pred
        
        rmse_scores[i] = mean_squared_error(np.log1p(test_y), np.log1p(y_pred), squared=False)
        rmse_total += rmse_scores[i]

    # Final model and the final model's score
    rmse_average = rmse_total / cv_amount
    rmse_list = []
    for i in range(len(rmse_scores)):
        rmse_list.append(rmse_scores[i])
    max_rmse = max(rmse_list)
    min_rmse = min(rmse_list)
    index_max_rmse = find_index(rmse_list, max_rmse)
    index_min_rmse = find_index(rmse_list, min_rmse)
    final_model = models['model_{}'.format(index_max_rmse)]
    display(final_model.summary())
    
    # Residual analysis
    #residuals_cv = np.expm1(folds_y[index_min_r2]) - predictions[index_min_r2]
    #plot_residuals(residuals_cv, predictions[index_min_r2])
    #normality(residuals_cv)
    #print(residuals_cv.sort_values())
    print("-----------------------------------------------------------------------------")
    print("All RMSE score:  ", rmse_scores)
    print("Max RMSE score:  ", rmse_scores[index_max_rmse], index_max_rmse)
    print("Min RMSE score:  ", rmse_scores[index_min_rmse], index_min_rmse)
    print("Average RMSE score:  ", rmse_average)
    return final_model, rmse_average

In [7]:
def cyclic_features(df, df_):
    df['date_decision'] = pd.to_datetime(df['date_decision'])
    
    days_in_year = 365.25
    df['day_of_year'] = df['date_decision'].dt.dayofyear
    df['year_sin'] = np.sin(2 * np.pi * df['day_of_year'] / days_in_year)
    df['year_cos'] = np.cos(2 * np.pi * df['day_of_year'] / days_in_year)
    
    days_in_month = 30.437
    df['day_of_month'] = df['date_decision'].dt.day
    df['month_sin'] = np.sin(2 * np.pi * df['day_of_month'] / days_in_month)
    df['month_cos'] = np.cos(2 * np.pi * df['day_of_month'] / days_in_month)
    
    days_in_week = 7
    df['day_of_week'] = df['date_decision'].dt.dayofweek
    df['week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / days_in_week)
    df['week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / days_in_week)
    

    df_['date_decision'] = pd.to_datetime(df_['date_decision'])
    
    days_in_year = 365.25
    df_['day_of_year'] = df_['date_decision'].dt.dayofyear
    df_['year_sin'] = np.sin(2 * np.pi * df_['day_of_year'] / days_in_year)
    df_['year_cos'] = np.cos(2 * np.pi * df_['day_of_year'] / days_in_year)
    
    days_in_month = 30.437
    df_['day_of_month'] = df_['date_decision'].dt.day
    df_['month_sin'] = np.sin(2 * np.pi * df_['day_of_month'] / days_in_month)
    df_['month_cos'] = np.cos(2 * np.pi * df_['day_of_month'] / days_in_month)
    
    days_in_week = 7
    df_['day_of_week'] = df_['date_decision'].dt.dayofweek
    df_['week_sin'] = np.sin(2 * np.pi * df_['day_of_week'] / days_in_week)
    df_['week_cos'] = np.cos(2 * np.pi * df_['day_of_week'] / days_in_week)
    return df, df_

In [9]:
train_path = '/kaggle/input/home-credit-credit-risk-model-stability/csv_files/train/train_base.csv'
test_path = '/kaggle/input/home-credit-credit-risk-model-stability/csv_files/test/test_base.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train, test = cyclic_features(train, test)

selected_features = ['MONTH', 'WEEK_NUM', 'year_sin', 'year_cos', 'month_sin', 'month_cos', 'week_sin', 'week_cos']

In [10]:
imputer = SimpleImputer(strategy='constant', fill_value=0)
train[selected_features] = imputer.fit_transform(train[selected_features])

test = preprocess_data(test, selected_features, imputer)

y = train['target']
X = train[selected_features]
X_test = test[selected_features]

folds_X = create_k_folds(X, fold_n=5)
folds_y = create_k_folds(y, fold_n=5)

In [11]:
final_model, rmse_average = train_final_model(folds_X, folds_y)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     102.7
Date:                Tue, 07 May 2024   Prob (F-statistic):          4.58e-172
Time:                        18:08:13   Log-Likelihood:             4.3120e+05
No. Observations:             1221328   AIC:                        -8.624e+05
Df Residuals:                 1221319   BIC:                        -8.623e+05
Df Model:                           8                                         
Covariance Type:                  HC0                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         16.7325      2.279      7.341      0.000      12.265      21.200
x1         -8.274e-05   1.13e-05     -7.328      0.000      -0.000   -6.06e-05
x2             0.0002   2.14e-05      8.028      0.000       0.000       0.000
x3             0.0033      0.000      9.197      0.000       0.003       0.004
x4             0.0054      0.000     23.564      0.000       0.005       0.006
x5            -0.0007      0.000     -3.347      0.001      -0.001      -0.000
x6            -0.0002      0.000     -0.774      0.439      -0.001       0.000
x7             0.0032      0.000     14.411      0.000       0.003       0.004
x8            -0.0004      0.000     -1.901      0.057      -0.001    1.27e-05
==============================================================================
Omnibus:                  1263783.541   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         47660848.198
Skew:                           5.524   Prob(JB):                         0.00
Kurtosis:                      31.540   Cond. No.                     2.84e+09
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC0)
[2] The condition number is large, 2.84e+09. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

-----------------------------------------------------------------------------
All RMSE score:   {0: 0.12304632346915169, 1: 0.11844053209446963, 2: 0.11319893392654351, 3: 0.13259280614825095, 4: 0.1185761993225907}
Max RMSE score:   0.13259280614825095 3
Min RMSE score:   0.11319893392654351 2
Average RMSE score:   0.12117095899220129


In [12]:
model = train_model_statsmodels(X, y)

X = sm.add_constant(X)
y_pred = model.predict(X)
rmse_score = mean_squared_error(np.log1p(y), np.log1p(y_pred), squared=False)
print("RMSE", rmse_score)

RMSE 0.12125306445809153


In [13]:
X_test = sm.add_constant(X_test, has_constant='add')
predicted_log_score = model.predict(X_test)
predicted_score = np.expm1(predicted_log_score)

In [14]:
predicted_score_df = pd.DataFrame({
    'case_id': test['case_id'].to_numpy(),
    'score': predicted_score
})
predicted_score_df.to_csv('submission.csv', index=False)

In [15]:
predicted_score_df

,case_id,score
0,57543,0.027669
1,57549,0.037934
2,57551,0.034644
3,57552,0.034644
4,57569,0.037428
5,57630,0.038188
6,57631,0.022142
7,57632,0.032760
8,57633,0.040983
9,57634,0.041975
